In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import requests
import json
import configparser
import unittest
import uuid
import datetime

In [2]:
CONFIG_FILE = 'config.ini'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

In [3]:
def unit_test(testClassInstance):
    unittest.main(testClassInstance, argv=['first-arg-is-ignored'], exit=False)

In [30]:
class DimHealthAPI():

    def __init__(self, api_server):
        self.api_server = api_server
        self.req = None
        self.token = ''

    def login(self, username, password):
        self.req = requests.post(
            self.api_server + '/rest_api_login/',
            headers={'content-type': 'application/json'},
            json={
                'username': username,
                'password': password
            }
        )

        if self.req.ok:
            self.token = json.loads(self.req.content)['token']
        else:
            self.err_details = self.req.content
            
        return self.req.ok

    def add_patients(self, patients):
        headers = {
           #'Content-Type': 'application/json',
           'Authorization': 'Token ' + self.token
        }
        self.req = requests.post(
            self.api_server + '/rest_api_add_patients/',
            headers=headers,
            json=patients
        )

        ok = self.req.ok # and json.loads(self.req.content)['status'] in (200,201)
        if not ok:
            self.err_details = self.req.content
            
        return ok    
    

# tests
dh_api = DimHealthAPI('http://test.dimhealth.ru:8054')
if not dh_api.login('1','2') and dh_api.err_details == b'{"non_field_errors":["Unable to log in with provided credentials."]}':
    print('TESTS PASSED')
else:
    print('TESTS FAIL')

TESTS PASSED


In [32]:
# Patient data exampe:
# {'uuid': '51ac59c2-3bf5-4866-b436-4a6d0e37h95c',
#  'conceptClass': 'base_knowledge',
#  'enc_type': 'risk2',
#  'fields': [{'date': '2019-02-2800:00:00',
#    'map': {'HEMOGLOBIN': 99, 'BLOOD SUGAR': 5, 'HB': 20, 'ALP': 82}},
#   {'date': '2019-03-26 00:00:00',
#    'map': {'HEMOGLOBIN': 110, 'BLOOD SUGAR': 8, 'HB': 18}},
#   {'date': '2019-01-2600:00:00',
#    'map': {'HEMOGLOBIN': 100, 'BLOOD SUGAR': 7, 'ALP': 25}}]}

class Patient:
    
    def __init__(self, uuid, pcode, procedures):
        '''
        Patient info
        
        params:
        - pcode: internal person id (from infoclinic)
        - procedures: list of Procedure() objects
        '''
        self.uuid = uuid
        self.pcode = pcode
        self.procedures = procedures # list of procedures
      
    def to_json(self):
        return {
            'uuid': self.uuid,
            'conceptClass': 'base_knowledge', # reserved
            'enc_type': 'risk2', # reserved
            'fields': list(map(lambda x: x.to_json(), self.procedures))
        }
    
class Procedure:
    
    def __init__(self, date, observations):
        '''
        Medical procedure
        
        params:
        - date: procedure datetime
        - observation: dictionary with medical observations (hemoglobin etc.) 
        '''
        self.date = date
        self.observations = observations
        
    def to_json(self):
        return {
            'date': self.date,
            'map': self.observations
        }
    

## =============> UNIT-TESTS <==============
class TestPatientData(unittest.TestCase):
    
    def __init__(self, *args, **kwargs):
        super(TestPatientData, self).__init__(*args, **kwargs)
        
        dt = datetime.datetime.strptime('2019-02-02 22:33:44', DATETIME_FORMAT)            
        observations = {
            'gender': 'M',
            'birthdate': '2000-01-01',
            'weight': 80
        }
        
        self.procedure = Procedure(dt.strftime(DATETIME_FORMAT), observations)
        self.test_json = {
            'date': '2019-02-02 22:33:44',
            'map': {
                'gender': 'M', 
                'birthdate': 
                '2000-01-01', 
                'weight': 80
            }
        }
        
    def test_procedure(self):        
        self.assertEqual(self.procedure.to_json(), self.test_json)

    def test_patient(self):        
        uuid = '5d11d14e-ae0d-427a-b341-526d6c8d0b0c'
        patient = Patient(uuid, '007', [self.procedure])
        patient_json = {
            'uuid': uuid,
            'conceptClass': 'base_knowledge',
            'enc_type': 'risk2',
            'fields': [
                {
                    'date': '2019-02-02 22:33:44',
                    'map': {
                        'gender': 'M',
                        'birthdate': '2000-01-01',
                        'weight': 80
                    }
                }
            ]
        }
        self.assertEqual(patient.to_json(),patient_json)

unit_test(TestPatientData());

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


In [33]:
config = configparser.ConfigParser()
config.read(CONFIG_FILE)
api_config = config._sections['dim-health-api']

dim_health_api = DimHealthAPI(api_config['api_server'])
if not dim_health_api.login(api_config['username'], api_config['password']):
    print('Can\'t login to dimhealth:', dim_health_api.err_details)
else:
    print('Connection successfull')
    #dim_health_api.add_patients()

Connection successfull


In [41]:
patient_json = {'patients': [
    {
            'uuid': str(uuid.uuid4()),
            'conceptClass': 'base_knowledge',
            'enc_type': 'risk2',
            'fields': [
                {
                    'date': '2019-02-02 22:33:44',
                    'map': {
                        'gender': 'M',
                         'birthdate': '2000-01-01',
                         'weight': 80
                    }
                }
            ]
        }
]}

if not dim_health_api.add_patients(patient_json):
    print(dim_health_api.err_details)

In [42]:
dim_health_api.req.content

b'{"patients": [{"uuid": "29974e5b-611c-47c0-8db9-c63cbe08894b", "fields": [{"date": "2019-02-02 22:33:44", "result": {"status": 200, "message": "\\u0441\\u043e\\u0445\\u0440\\u0430\\u043d\\u0435\\u043d\\u043e"}}]}]}'

In [169]:
#json.dumps(patient_json)
patient_json

{'uuid': 'b3c67058-962f-4bc2-a752-13ae92e6d035',
 'conceptClass': 'base_knowledge',
 'enc_type': 'risk2',
 'fields': [{'date': '2019-02-02 22:33:44', 'map': {'gender': 'M'}}]}

In [ ]:
#mapping infoclinic to dimhealth
# TotalHol           Общий холестерин, ммоль/л
# hol_lpnp           Холестерин ЛПНП, ммоль/л
# hol_lpvp           Холестерин ЛПВП, ммоль/л
# triglycerides      Триглицериды, ммоль/л
# blood_clqotting    МНО, ед.
# creatinin20        Суточная экскреция креатинина, мг/сутки
# albumin_day        Суточная экскреция альбумина, мг/сутки
# albumin            Концентрация альбумина в моче, мг/л
# speed_club_filtrac Скорость клубочковой фильтрации, мд/мин/1,73м2
# citokeens          Повышенный уровень цитокинов

mapping = [
    ['gender', lambda x: 'M' if x['gender'] == 1 else 2],
    ['bitdhdate', lamba x: x['treatdate'] - relativedelta(years=x['age'])],
    ['TotalHol', 'Холестерин общий'],
    ['hol_lpnp', 'Холестерин ЛПВП'],
    ['hol_lpvp','Холестерин ЛПНП (LDL)'],
    ['triglycerides','Триглицериды'],
    ['blood_clqotting','Международное нормализованное отношение (МНО)'],
    ['albumin','Альбумин']
]


In [181]:
import pandas as pd
df = pd.read_csv('data/lab_2018_1.csv', encoding='utf-8', delimiter='\t')

In [222]:
df[df.DGNAME.fillna('').str.contains('акн')].DGNAME.value_counts()

Series([], Name: DGNAME, dtype: int64)

In [218]:
#df.F25_NAMEPARAMS.value_counts().to_excel('f25.xls')
df.DGNAME.value_counts().to_excel('dgname.xls')

In [176]:
d = datetime.datetime.today()
datetime.datetime.year

from dateutil.relativedelta import relativedelta
new_date = old_date - relativedelta(years=x)

<attribute 'year' of 'datetime.date' objects>

In [223]:
# git inital setup
# !git init
# !git config --global user.email "pomka@yandex.ru"
# !git config --global user.name "Roman Shekhovtsov"
# !git remote add origin git@github.com:rshekhovtsov/dim-health.git

Инициализирован пустой репозиторий Git в /home/roma/Документы/lab-data/.git/


In [3]:
# commit & push
from IPython.display import Javascript

def notebook_save():
    script = '''
    require(["base/js/namespace"],function(Jupyter) {
        Jupyter.notebook.save_checkpoint();
    });
    '''
    Javascript(script)
    print('This notebook has been saved')

notebook_save()

!git add *.ipynb
!git add *.xls
!git commit -m "1st cell set width 100%"
!git push origin master

# !git add *.docx
# !git commit -m "api doc added"
# !git push origin master


This notebook has been saved


The file will have its original line endings in your working directory


[master 97db972] 1st cell set width 100%
 Committer: РЁРµС…РѕРІС†РѕРІ Р РѕРјР°РЅ <rshekhovtsov@ent.drclinics.ru>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 1 file changed, 66 insertions(+), 13 deletions(-)


To github.com:rshekhovtsov/dim-health.git
   a81f7b8..97db972  master -> master
